In [3]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug, LabValue
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter


DB Password: ········


In [4]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()
Pulse_Case = pq.read_table('Cohorts/Case/Pulse_Rate_cases.parquet').to_pandas()
Pulse_Control = pq.read_table('Cohorts/Control/Pulse_Rate_control.parquet').to_pandas()

In [6]:
Case_filtered_15_540.index

Index(['100067499', '100091035', '1004105957', '100423963', '1005188482',
       '1006954100', '1006987597', '1007749798', '1007977434', '1008498682',
       ...
       '988085702', '988232727', '992976555', '993560490', '993881441',
       '994109921', '998439122', '998462024', '998794413', '999008531'],
      dtype='object', name='medical_record_number', length=131292)

In [3]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [ ]:
Case_filtered_15_540['year_of_birth'].plot.hist().grid()
plt.axis([-1, 3.5, 0, 80000])
#plt.xlabel("Gender", labelpad=5)
#plt.ylabel("Count of People", labelpad=14)
#plt.title("Gender of cases with HT", y=1.02)
plt.show()

In [77]:
y = Control_filtered_15_540
y = df_to_cohort(y)

In [78]:
y

In [79]:
Carbon = y.get(LabValue("%CARBON%DIOXIDE-BLD%"))

Fetching data for LabValue (...)


In [8]:
C_merged = Carbon.merge(Case_filtered_15_540[["HT_Onset"]], left_on="medical_record_number", right_index=True, how="left")

In [10]:
C_filtered = C_merged.loc[(C_merged["HT_Onset"] - C_merged["age_in_days"] > 180)]

In [17]:
def filter_df_Onset(to_check, df_with_HT):
    final_merged = to_check.merge(df_with_HT[["HT_Onset"]], left_on="medical_record_number", right_index=True, how="left")
    final_filtered = final_merged.loc[(final_merged["HT_Onset"] - final_merged["age_in_days"]) > 180]
    return final_filtered

In [18]:
C = filter_df_Onset(Carbon, Case_filtered_15_540)

In [82]:
#for controls
C = Carbon

In [81]:
C

,CARBON_DIOXIDE_BLD_min,CARBON_DIOXIDE_BLD_max,CARBON_DIOXIDE_BLD_mean
medical_record_number,,,
1000054185,20.8,24.9,22.500000
1000083464,27.7,27.7,27.700000
1000124322,28.7,28.7,28.700000
1000129011,21.0,28.1,24.025000
1000133396,26.7,27.2,26.866667
1000134906,27.4,27.4,27.400000
1000165222,19.8,25.9,23.750000
1000172084,27.3,27.3,27.300000
1000199143,26.0,28.2,27.033333


In [46]:
C_min = C.groupby([C["medical_record_number"]])['numeric_value'].min().to_frame()

In [47]:
C_min.rename(columns={"numeric_value": "CARBON_DIOXIDE_BLD_min"}, inplace=True)

In [48]:
C_min

,CARBON_DIOXIDE_BLD_min
medical_record_number,
1000096552,23.0
1000173142,27.4
1000227914,19.0
1000271684,16.3
1000273082,20.0
1000434381,20.0
1000546245,15.5
1000604785,25.7
1000628903,26.8


In [60]:
def min_max_mean(df, lab_value_name):
    df_min = df.groupby([df["medical_record_number"]])['numeric_value'].min().to_frame()
    df_max = df.groupby([df["medical_record_number"]])['numeric_value'].max().to_frame()
    df_mean = df.groupby([df["medical_record_number"]])['numeric_value'].mean().to_frame()
    df_min.rename(columns={"numeric_value": lab_value_name+"_min"}, inplace=True)
    df_max.rename(columns={"numeric_value": lab_value_name+"_max"}, inplace=True)
    df_mean.rename(columns={"numeric_value": lab_value_name+"_mean"}, inplace=True)
    result = df_min.merge(df_max, left_index=True, right_index=True)
    result = result.merge(df_mean, left_index=True, right_index=True)
    return result

In [83]:
CARBON_DIOXIDE_BLD = min_max_mean(C, "CARBON_DIOXIDE_BLD")

In [84]:
CARBON_DIOXIDE_BLD

,CARBON_DIOXIDE_BLD_min,CARBON_DIOXIDE_BLD_max,CARBON_DIOXIDE_BLD_mean
medical_record_number,,,
1000054185,20.8,24.9,22.500000
1000083464,27.7,27.7,27.700000
1000124322,28.7,28.7,28.700000
1000129011,21.0,28.1,24.025000
1000133396,26.7,27.2,26.866667
1000134906,27.4,27.4,27.400000
1000165222,19.8,25.9,23.750000
1000172084,27.3,27.3,27.300000
1000199143,26.0,28.2,27.033333


In [86]:
CARBON_DIOXIDE_BLD.to_parquet("Cohorts/Control/LabValues/Control_CARBON_DIOXIDE_BLD.parquet")

In [33]:
Carbon_filtered = C.drop_duplicates(['medical_record_number'])

In [34]:
Carbon_filtered

,medical_record_number,age_in_days,test_name,abnormal_flag,result_flag,numeric_value,unit_of_measurement,HT_Onset
8,2296580641,12111,CARBON DIOXIDE-BLD,False,,32.0,MEQ/L,13480.0
28,4100160124,17460,CARBON DIOXIDE-BLD,True,,30.8,MEQ/L,18005.0
30,1364769039,11512,CARBON DIOXIDE-BLD,False,,27.0,MEQ/L,13689.0
34,1838244068,26349,CARBON DIOXIDE-BLD,True,,25.6,MEQ/L,27348.0
62,1657015815,23458,CARBON DIOXIDE-BLD,False,,27.3,MEQ/L,25417.0
71,930827849,18509,CARBON DIOXIDE-BLD,True,HIGH,32.1,MEQ/L,19268.0
85,3661980535,25654,CARBON DIOXIDE-BLD,False,,23.0,MEQ/L,26067.0
91,229285665,14550,CARBON DIOXIDE-BLD,True,,26.9,MEQ/L,15141.0
94,2894372402,26364,CARBON DIOXIDE-BLD,True,,28.6,MEQ/L,31553.0
95,720831128,15129,CARBON DIOXIDE-BLD,True,,22.3,MEQ/L,15344.0


In [87]:
Hematocrit = y.get(LabValue("%HEMATOCRIT%"))

Fetching data for LabValue (...)


In [22]:
H = filter_df_Onset(Hematocrit, Case_filtered_15_540)

In [88]:
H = Hematocrit

In [89]:
HEMATOCRIT = min_max_mean(H, "HEMATOCRIT")

In [90]:
HEMATOCRIT

,HEMATOCRIT_min,HEMATOCRIT_max,HEMATOCRIT_mean
medical_record_number,,,
1000004097,30.1,34.8,32.450000
100002558,42.0,42.0,42.000000
1000044143,36.9,36.9,36.900000
1000054185,32.2,35.3,34.260000
1000083464,48.4,48.4,48.400000
1000124322,33.2,33.2,33.200000
1000129011,30.0,39.5,36.203333
1000133396,39.1,47.4,42.566667
1000134906,36.6,37.0,36.866667


In [91]:
HEMATOCRIT.to_parquet("Cohorts/Control/LabValues/Control_HEMATOCRIT.parquet")

In [24]:
Hematocrit_filtered = H.drop_duplicates(['medical_record_number'])

In [25]:
Hematocrit_filtered

,medical_record_number,age_in_days,test_name,abnormal_flag,result_flag,numeric_value,unit_of_measurement,HT_Onset
8,1658499582,10656,HEMATOCRIT,True,,43.5,%,11233.0
17,1535391506,23950,HEMATOCRIT,True,LOW,32.2,%,26504.0
71,253001709,8985,HEMATOCRIT,True,LOW,24.2,%,10263.0
89,2703054054,20862,HEMATOCRIT,True,,43.6,%,24263.0
90,632977624,16414,HEMATOCRIT,True,,37.7,%,16782.0
92,2291139209,21005,HEMATOCRIT,True,LOW,30.2,%,21402.0
102,219515074,15026,HEMATOCRIT,True,LOW,22.9,%,15228.0
110,2941203450,20209,"HEMATOCRIT , VEN",True,,40.0,%,20931.0
136,1809535780,11763,HEMATOCRIT,True,,34.8,%,13862.0
152,913324034,4154,HEMATOCRIT,True,,40.1,%,5382.0


In [26]:
meds = Carbon["medical_record_number"]

In [27]:
x = Hematocrit_filtered[Hematocrit_filtered.medical_record_number.isin(meds)]

In [92]:
Glucose = y.get(LabValue("%GLUCOSE%"))

Fetching data for LabValue (...)


In [65]:
X = filter_df_Onset(Glucose, Case_filtered_15_540)
GLUCOSE = min_max_mean(X, "GLUCOSE")


In [93]:
GLUCOSE = min_max_mean(Glucose, "GLUCOSE")

In [94]:
GLUCOSE

,GLUCOSE_min,GLUCOSE_max,GLUCOSE_mean
medical_record_number,,,
100002558,102.0,102.0,102.000000
1000044143,91.0,91.0,91.000000
1000054185,95.0,115.0,105.750000
1000064392,150.0,150.0,150.000000
1000083464,116.0,116.0,116.000000
1000124322,95.0,95.0,95.000000
1000129011,73.0,133.0,94.000000
1000133396,93.0,116.0,101.000000
1000134906,71.0,85.0,78.000000


In [96]:
GLUCOSE.to_parquet("Cohorts/Control/LabValues/Control_GLUCOSE.parquet")

In [97]:
Creatinine = y.get(LabValue("%CREATININE-SERUM%"))

Fetching data for LabValue (...)


In [99]:
X = filter_df_Onset(Creatinine, Case_filtered_15_540)
CREATININE = min_max_mean(X, "CREATININE-SERUM")

In [101]:
CREATININE = min_max_mean(Creatinine, "CREATININE-SERUM")

In [102]:
CREATININE

,CREATININE-SERUM_min,CREATININE-SERUM_max,CREATININE-SERUM_mean
medical_record_number,,,
100002558,0.90,0.90,0.900000
1000044143,0.62,0.62,0.620000
1000054185,0.40,0.70,0.550000
1000083464,0.97,0.97,0.970000
1000124322,0.91,0.91,0.910000
1000129011,0.40,0.89,0.598000
1000133396,0.60,0.70,0.666667
1000134906,0.50,0.50,0.500000
1000144873,0.95,0.95,0.950000


In [103]:
CREATININE.to_parquet("Cohorts/Control/LabValues/Control_CREATININE.parquet")

In [104]:
Urea = y.get(LabValue("%UREA%NITROGEN-BLD%"))

Fetching data for LabValue (...)


In [105]:
X = filter_df_Onset(Urea, Case_filtered_15_540)
UREA = min_max_mean(Urea, "UREA_NITROGEN-BLD")

In [106]:
UREA

,UREA_NITROGEN-BLD_min,UREA_NITROGEN-BLD_max,UREA_NITROGEN-BLD_mean
medical_record_number,,,
100002558,17.0,17.0,17.000000
1000044143,13.0,13.0,13.000000
1000054185,6.0,12.0,9.500000
1000083464,17.0,17.0,17.000000
1000124322,20.0,20.0,20.000000
1000129011,7.0,14.0,10.000000
1000133396,10.0,11.0,10.666667
1000134906,11.0,11.0,11.000000
1000144873,14.0,14.0,14.000000


In [108]:
UREA.to_parquet("Cohorts/Control/LabValues/Control_UREA.parquet")